## Get Requirements

In [3]:
import torch
from torchvision import transforms, datasets
import torch.nn as nn
from torch import optim as optim
import numpy as np
import matplotlib.pyplot as plt
import h5py
#import helpers
import numpy as np
from pathlib import Path
import torch
from torch.utils import data
# import cv2
import math
from PIL import Image
import random
from torch.utils.data import Dataset, DataLoader
import time
import glob
import scipy.signal
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
print(torch.__version__)


from datasets.sleep_edf import split_data, SleepEDF_MultiChan_Dataset
from datasets.sleep_edf import SleepEDF_Seq_MultiChan_Dataset_Main as SleepEDF_Seq_MultiChan_Dataset
from models.epoch_cmt import Epoch_Cross_Transformer_Network
from models.sequence_cmt import Seq_Cross_Transformer_Network 
from utils.metrics import accuracy, kappa, g_mean, plot_confusion_matrix, confusion_matrix, AverageMeter 

1.11.0+cu113


In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device 
project_path = "output"
if not os.path.isdir(project_path):
        os.makedirs(project_path)
        print(f"Project directory created at {project_path}")

## Data

In [6]:
train_data_list = [0,1,2,3]  
val_data_list = [4]  
path = r"C:\Users\Tian_Yumi\Documents\GitHub\CMT-Sleep\data_preparations\extract_dataset_multi_epoch"

eeg_list = glob.glob(f'{path}/x*.h5')
eeg_list.sort()
[train_eeg_list, val_eeg_list] = split_data(eeg_list,train_data_list,val_data_list)
print(train_eeg_list)
print(val_eeg_list)

mean_eeg_list = glob.glob(f'{path}/mean*.h5')
mean_eeg_list.sort()
[train_mean_eeg_list, val_mean_eeg_list] = split_data(mean_eeg_list,train_data_list,val_data_list)
print(train_mean_eeg_list)
print(val_mean_eeg_list)

sd_eeg_list = glob.glob(f'{path}/std*.h5')
sd_eeg_list.sort()
[train_sd_eeg_list, val_sd_eeg_list] = split_data(sd_eeg_list,train_data_list,val_data_list)
print(train_sd_eeg_list)
print(val_sd_eeg_list)

#########################################################################################################################

eog_list = glob.glob(f'{path}/eog*.h5')
eog_list.sort()
[train_eog_list, val_eog_list] = split_data(eog_list,train_data_list,val_data_list)
print(train_eog_list)
print(val_eog_list)

mean_eog_list = glob.glob(f'{path}/eog_m*.h5')
mean_eog_list.sort()
[train_mean_eog_list, val_mean_eog_list] = split_data(mean_eog_list,train_data_list,val_data_list)
print(train_mean_eog_list)
print(val_mean_eog_list)

sd_eog_list = glob.glob(f'{path}/eog_std*.h5')
sd_eog_list.sort()
[train_sd_eog_list, val_sd_eog_list] = split_data(sd_eog_list,train_data_list,val_data_list)
print(train_sd_eog_list)
print(val_sd_eog_list)


# 
# eeg2_list = glob.glob(f'{path}/eeg*.h5')
# eeg2_list.sort()
# [train_eeg2_list, val_eeg2_list] = split_data(eeg2_list,train_data_list,val_data_list)
# print(train_eeg2_list)
# print(val_eeg2_list)
# 
# mean_eeg2_list = glob.glob(f'{path}/eeg_m*.h5')
# mean_eeg2_list.sort()
# [train_mean_eeg2_list, val_mean_eeg2_list] = split_data(mean_eeg2_list,train_data_list,val_data_list)
# print(train_mean_eeg2_list)
# print(val_mean_eeg2_list)
# 
# sd_eeg2_list = glob.glob(f'{path}/eeg_s*.h5')
# sd_eeg2_list.sort()
# [train_sd_eeg2_list, val_sd_eeg2_list] = split_data(sd_eeg2_list,train_data_list,val_data_list)
# print(train_sd_eeg2_list)
# print(val_sd_eeg2_list)



label_list = glob.glob(f'{path}/y*.h5')
label_list.sort()
[train_label_list, val_label_list] = split_data(label_list,train_data_list,val_data_list)
print(train_label_list)
print(val_label_list)


print(val_data_list)

['C:\\Users\\Tian_Yumi\\Documents\\GitHub\\CMT-Sleep\\data_preparations\\extract_dataset_multi_epoch\\x1.h5'
 'C:\\Users\\Tian_Yumi\\Documents\\GitHub\\CMT-Sleep\\data_preparations\\extract_dataset_multi_epoch\\x2.h5'
 'C:\\Users\\Tian_Yumi\\Documents\\GitHub\\CMT-Sleep\\data_preparations\\extract_dataset_multi_epoch\\x3.h5'
 'C:\\Users\\Tian_Yumi\\Documents\\GitHub\\CMT-Sleep\\data_preparations\\extract_dataset_multi_epoch\\x4.h5']
['C:\\Users\\Tian_Yumi\\Documents\\GitHub\\CMT-Sleep\\data_preparations\\extract_dataset_multi_epoch\\x5.h5']
['C:\\Users\\Tian_Yumi\\Documents\\GitHub\\CMT-Sleep\\data_preparations\\extract_dataset_multi_epoch\\mean1.h5'
 'C:\\Users\\Tian_Yumi\\Documents\\GitHub\\CMT-Sleep\\data_preparations\\extract_dataset_multi_epoch\\mean2.h5'
 'C:\\Users\\Tian_Yumi\\Documents\\GitHub\\CMT-Sleep\\data_preparations\\extract_dataset_multi_epoch\\mean3.h5'
 'C:\\Users\\Tian_Yumi\\Documents\\GitHub\\CMT-Sleep\\data_preparations\\extract_dataset_multi_epoch\\mean4.h5']
['C:

In [7]:
num_seq = 5
train_dataset = SleepEDF_Seq_MultiChan_Dataset(eeg_file = train_eeg_list , 
                                           eog_file = train_eog_list, 
                                           label_file = train_label_list, 
                                           device = device, mean_eeg_l = train_mean_eeg_list, sd_eeg_l = train_sd_eeg_list, 
                                           mean_eog_l = train_mean_eog_list, sd_eog_l = train_sd_eog_list, 
                                           sub_wise_norm = True,
                                           num_seq = num_seq,
                                           transform=transforms.Compose([
                                               transforms.ToTensor(),
                                                ]) )

val_dataset = SleepEDF_Seq_MultiChan_Dataset(eeg_file = val_eeg_list ,
                                         eog_file = val_eog_list, 
                                         label_file = val_label_list, 
                                         device = device, mean_eeg_l = val_mean_eeg_list, sd_eeg_l = val_sd_eeg_list,
                                         mean_eog_l = val_mean_eog_list, sd_eog_l = val_sd_eog_list,
                                         sub_wise_norm = True, num_seq = num_seq,
                                         transform=transforms.Compose([
                                               transforms.ToTensor(),
                                                ]) )

Reading from C:\Users\Tian_Yumi\Documents\GitHub\CMT-Sleep\data_preparations\extract_dataset_multi_epoch\x1.h5 ====================================================
Keys in the h5py file : <KeysViewHDF5 ['data']>
Number of samples : 2022
Shape of each data : (2022, 5, 3000)
Reading from C:\Users\Tian_Yumi\Documents\GitHub\CMT-Sleep\data_preparations\extract_dataset_multi_epoch\eog1.h5 ====================================================
Keys in the h5py file : <KeysViewHDF5 ['data']>
Number of samples : 2022
Shape of each data : (2022, 5, 3000)
Reading from C:\Users\Tian_Yumi\Documents\GitHub\CMT-Sleep\data_preparations\extract_dataset_multi_epoch\y1.h5 ====================================================
Keys in the h5py file : <KeysViewHDF5 ['data']>
Number of samples : 2022
Shape of each data : (2022, 5)
Reading from C:\Users\Tian_Yumi\Documents\GitHub\CMT-Sleep\data_preparations\extract_dataset_multi_epoch\x2.h5 ====================================================
Keys in the h5py f

In [8]:
batch_size = 8
train_data_loader = data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
val_data_loader = data.DataLoader(val_dataset, batch_size = batch_size, shuffle = True)

In [9]:
eeg_data, eog_data, label = next(iter(train_data_loader))
print(f"EEG batch shape: {eeg_data.size()}")
print(f"EOG batch shape: {eog_data.size()}")
# print(f"EMG batch shape: {eeg2_data.size()}")
print(f"Labels batch shape: {label.size()}")

# eeg_data_temp = torch.reshape(eeg_data[0],(1,eeg_data[0].shape[1]*eeg_data[0].shape[2]))
# eog_data_temp = torch.reshape(eog_data[0],(1,eog_data[0].shape[1]*eog_data[0].shape[2]))
# 
# t = np.arange(0,30,1/100)
# plt.figure(figsize = (10,10))
# plt.plot(eeg_data_temp[0].squeeze())
# plt.plot(eog_data_temp[0].squeeze()+5)
# # plt.plot(t,eeg2_data[0].squeeze()+10)
# plt.title(f"Label {label[0].squeeze()}")
# plt.show()

eeg_data, eog_data, label = next(iter(val_data_loader))
print(f"EEG batch shape: {eeg_data.size()}")
print(f"EOG batch shape: {eog_data.size()}")
# print(f"EMG batch shape: {eeg2_data.size()}")
print(f"Labels batch shape: {label.size()}")
# 
# eeg_data_temp = torch.reshape(eeg_data[0],(1,eeg_data[0].shape[1]*eeg_data[0].shape[2]))
# eog_data_temp = torch.reshape(eog_data[0],(1,eog_data[0].shape[1]*eog_data[0].shape[2]))
# 
# # t = np.arange(0,30,1/100)
# plt.figure(figsize = (10,10))
# plt.plot(eeg_data_temp[0].squeeze())
# plt.plot(eog_data_temp[0].squeeze()+5)
# # plt.plot(t,eeg2_data[0].squeeze()+10)
# plt.title(f"Label {label[0].squeeze()}")
# plt.show()


print(f"EEG Minimum :{eeg_data.min()}")
print(f"EEG Maximum :{eeg_data.max()}")
print(f"EOG Minimum :{eog_data.min()}")
print(f"EOG Maximum :{eog_data.max()}")
# print(f"EMG Minimum :{eeg2_data.min()}")
# print(f"EMG Maximum :{eeg2_data.max()}")


print(f"EEG Mean :{torch.mean(eeg_data)}")
print(f"EEG Standard Deviation :{torch.std(eeg_data)}")
print(f"EOG Mean :{torch.mean(eog_data)}")
print(f"EOG Standard Deviation :{torch.std(eog_data)}")
# print(f"EMG Mean :{torch.mean(eeg2_data)}")
# print(f"EMG Standard Deviation :{torch.std(eeg2_data)}")



EEG batch shape: torch.Size([8, 3000, 5, 5])
EOG batch shape: torch.Size([8, 3000, 5, 5])
Labels batch shape: torch.Size([8, 5, 5])
EEG batch shape: torch.Size([8, 3000, 5, 5])
EOG batch shape: torch.Size([8, 3000, 5, 5])
Labels batch shape: torch.Size([8, 5, 5])
EEG Minimum :-6.710402466983561
EEG Maximum :7.445122580911266
EOG Minimum :-10.94195434791585
EOG Maximum :10.3628641533131
EEG Mean :6.997932492580954e-05
EEG Standard Deviation :1.1425711498208253
EOG Mean :-0.00013029404963406247
EOG Standard Deviation :0.9744379567369932


### Classification Model Sequence Cross-Modal Transformer

In [ ]:
import torch.optim as optim

d_model = 256 
dim_feedforward=1024  #
window_size = 50#25 50
Net = Seq_Cross_Transformer_Network(d_model = d_model, dim_feedforward=dim_feedforward,
                                window_size = window_size ).to(device)
# Net = torch.load('/home/jathu/fyp_g15_sleep_monitoring/Experiments/Sleep_edfx/V2_Seq_CMT/V2SEQ-32/checkpoint_model_best_kappa.pth.tar').to(device)

lr = 0.001#0.001
beta_1 =  0.9    
beta_2 =  0.999    
eps = 1e-9
n_epochs = 1000
weights = torch.tensor([1., 2., 1., 2., 2.]) 
print(f"weights: {weights}")
criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.Adam(Net.parameters(), lr=lr, betas=(beta_1, beta_2),eps = eps, weight_decay = 0.0001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5) 

pred= Net(eeg_data.float().to(device), eog_data.float().to(device))
print(len(pred),pred[0].shape)#,cls_outs.shape,len(feat_list))

### Training

In [ ]:
# Training the model
best_val_acc = 0
best_val_kappa = 0
for epoch_idx in range(n_epochs):  # loop over the dataset multiple times
    
    Net.train()
    print(f'===========================================================Training Epoch : [{epoch_idx+1}/{n_epochs}] ===========================================================================================================>')
    batch_time = AverageMeter()
    data_time = AverageMeter()
    
    losses = AverageMeter()
    val_losses = AverageMeter()
    
    train_accuracy = AverageMeter()
    val_accuracy = AverageMeter()

    train_sensitivity = AverageMeter()
    val_sensitivity = AverageMeter()
    
    train_specificity = AverageMeter()
    val_specificity = AverageMeter()

    train_gmean = AverageMeter()
    val_gmean = AverageMeter()

    train_kappa = AverageMeter()
    val_kappa = AverageMeter()

    train_f1_score = AverageMeter()
    val_f1_score = AverageMeter()

    train_precision = AverageMeter()
    val_precision = AverageMeter()

    class1_sens = AverageMeter()
    class2_sens = AverageMeter()
    class3_sens = AverageMeter()
    class4_sens = AverageMeter()
    class5_sens = AverageMeter()

    class1_spec = AverageMeter()
    class2_spec = AverageMeter()
    class3_spec = AverageMeter()
    class4_spec = AverageMeter()
    class5_spec = AverageMeter()

    class1_f1 = AverageMeter()
    class2_f1 = AverageMeter()
    class3_f1 = AverageMeter()
    class4_f1 = AverageMeter()
    class5_f1 = AverageMeter()

    end = time.time()

    for batch_idx, data_input in enumerate(train_data_loader):
        # get the inputs; data is a list of [inputs, labels]
        data_time.update(time.time() - end)
        eeg,eog, labels = data_input
        cur_batch_size = len(eeg)
        optimizer.zero_grad()

        outputs = Net(eeg.float().to(device), eog.float().to(device))

        loss = 0
        for ep in range(num_seq):
          loss += criterion(outputs[ep].cpu(), labels[:,ep])


          train_accuracy.update(accuracy(outputs[ep].cpu(), labels[:,ep]))
          _,_,_,_,sens,spec,f1, prec = confusion_matrix(outputs[ep].cpu(), labels[:,ep], 5, cur_batch_size)
          train_sensitivity.update(sens)
          train_specificity.update(spec)
          train_f1_score.update(f1)
          train_precision.update(prec)
          train_gmean.update(g_mean(sens, spec))
          train_kappa.update(kappa(outputs[ep].cpu(), labels[:,ep]))
        
        loss.backward()
        optimizer.step()
        # scheduler.step()
        losses.update(loss)
        
       
        
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if batch_idx % 1000 == 0:
            msg = 'Epoch: [{0}/{3}][{1}/{2}]\t' \
                  'Train_Loss {loss.val:.5f} ({loss.avg:.5f})\t'\
                  'Train_Acc {train_acc.val:.5f} ({train_acc.avg:.5f})\t'\
                  'Train_Kappa {train_kap.val:.5f}({train_kap.avg:.5f})\t'\
                  'Train_MF1 {train_mf1.val:.5f}({train_mf1.avg:.5f})\t'\
                  'Train_G-Mean {train_gmean.val:.5f}({train_gmean.avg:.5f})\t'\
                  'Train_Precision {train_prec.val:.5f}({train_prec.avg:.5f})\t'\
                  'Train_Sensitivity {train_sens.val:.5f}({train_sens.avg:.5f})\t'\
                  'Train_Specificity {train_spec.val:.5f}({train_spec.avg:.5f})\t'\
                  'Time {batch_time.val:.3f}s ({batch_time.avg:.3f}s)\t' \
                  'Speed {speed:.1f} samples/s\t' \
                  'Data {data_time.val:.3f}s ({data_time.avg:.3f}s)\t'.format(
                      epoch_idx+1, batch_idx, len(train_data_loader),n_epochs, batch_time=batch_time,
                      speed=data_input[0].size(0)/batch_time.val,
                      data_time=data_time, loss=losses, train_acc = train_accuracy,
                      train_sens =train_sensitivity, train_spec = train_specificity, train_gmean = train_gmean,
                      train_kap = train_kappa, train_mf1 = train_f1_score, train_prec = train_precision)
            
            print(msg)


    #evaluation
    with torch.no_grad():
      Net.eval()
      for batch_val_idx, data_val in enumerate(val_data_loader):
        val_eeg,val_eog, val_labels = data_val
        cur_val_batch_size = len(val_eeg)
        pred = Net(val_eeg.float().to(device), val_eog.float().to(device))
        
        val_loss = 0
        for ep in range(num_seq):
          val_loss += criterion(pred[ep].cpu(), val_labels[:,ep])

          val_accuracy.update(accuracy(pred[ep].cpu(), val_labels[:,ep]))
          sens_list,spec_list,f1_list,prec_list, sens,spec,f1,prec = confusion_matrix(pred[ep].cpu(), val_labels[:,ep],  5, cur_val_batch_size)
          val_sensitivity.update(sens)
          val_specificity.update(spec)
          val_f1_score.update(f1)
          val_precision.update(prec)
          val_gmean.update(g_mean(sens, spec))
          val_kappa.update(kappa(pred[ep].cpu(), val_labels[:,ep]))

          class1_sens.update(sens_list[0])
          class2_sens.update(sens_list[1])
          class3_sens.update(sens_list[2])
          class4_sens.update(sens_list[3])
          class5_sens.update(sens_list[4])

          class1_spec.update(spec_list[0])
          class2_spec.update(spec_list[1])
          class3_spec.update(spec_list[2])
          class4_spec.update(spec_list[3])
          class5_spec.update(spec_list[4])

          class1_f1.update(f1_list[0])
          class2_f1.update(f1_list[1])
          class3_f1.update(f1_list[2])
          class4_f1.update(f1_list[3])
          class5_f1.update(f1_list[4])

        
        val_losses.update(val_loss)#.data.item())
        

      print(batch_val_idx)

     

      print(f'===========================================================Epoch : [{epoch_idx+1}/{n_epochs}]  Evaluation ===========================================================================================================>')
      print("Training Results : ")
      print(f"Training Loss     : {losses.avg}, Training Accuracy      : {train_accuracy.avg}, Training G-Mean      : {train_gmean.avg}") 
      print(f"Training Kappa      : {train_kappa.avg},Training MF1     : {train_f1_score.avg}, Training Precision      : {train_precision.avg}, Training Sensitivity      : {train_sensitivity.avg}, Training Specificity      : {train_specificity.avg}")
      
      print("Validation Results : ")
      print(f"Validation Loss   : {val_losses.avg}, Validation Accuracy : {val_accuracy.avg}, Validation G-Mean      : {val_gmean.avg}") 
      print(f"Validation Kappa     : {val_kappa.avg}, Validation MF1      : {val_f1_score.avg}, Validation Precision      : {val_precision.avg},  Validation Sensitivity      : {val_sensitivity.avg}, Validation Specificity      : {val_specificity.avg}")
    

      print(f"Class wise sensitivity W: {class1_sens.avg}, S1: {class2_sens.avg}, S2: {class3_sens.avg}, S3: {class4_sens.avg}, R: {class5_sens.avg}")
      print(f"Class wise specificity W: {class1_spec.avg}, S1: {class2_spec.avg}, S2: {class3_spec.avg}, S3: {class4_spec.avg}, R: {class5_spec.avg}")
      print(f"Class wise F1  W: {class1_f1.avg}, S1: {class2_f1.avg}, S2: {class3_f1.avg}, S3: {class4_f1.avg}, R: {class5_f1.avg}")

     

      if val_accuracy.avg > best_val_acc or (epoch_idx+1)%100==0 or val_kappa.avg > best_val_kappa:
          if val_accuracy.avg > best_val_acc:
           
            best_val_acc = val_accuracy.avg
            print("================================================================================================")
            print("                                          Saving Best Model (ACC)                                     ")
            print("================================================================================================")
            torch.save(Net, f'{project_path}/checkpoint_model_best_acc.pth.tar')
          if val_kappa.avg > best_val_kappa:
           
            best_val_kappa = val_kappa.avg
            print("================================================================================================")
            print("                                          Saving Best Model (Kappa)                                    ")
            print("================================================================================================")
            torch.save(Net, f'{project_path}/checkpoint_model_best_kappa.pth.tar')
          if (epoch_idx+1)%50==0:
            torch.save(Net, f'{project_path}/checkpoint_model_epoch_{epoch_idx+1}.pth.tar')
    lr_scheduler.step()
print('========================================Finished Training ===========================================')